In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models
import torch.nn.functional as F

# Data Augmentation and Normalization for the training set
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Normalization for the test set
transform_test = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Load the datasets
train_data = ImageFolder(root='Train', transform=transform_train)
test_data = ImageFolder(root='Test', transform=transform_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

# Load a pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Replace the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_data.classes))

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Implement a learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Step the scheduler
    scheduler.step()

    # Print loss for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test data: {accuracy:.2f}%')

# Save the trained model
torch.save(model.state_dict(), './skin_cancer_resnet.pth')


/home/sikandar/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sikandar/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/sikandar/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████████████████████████████████| 44.7M/44.7M [00:01<00:00, 41.7MB/s]


Epoch [1/20], Loss: 1.1638
Epoch [2/20], Loss: 0.6847
Epoch [3/20], Loss: 0.6961
Epoch [4/20], Loss: 0.3953
Epoch [5/20], Loss: 0.3637
Epoch [6/20], Loss: 0.2727
Epoch [7/20], Loss: 0.3051
Epoch [8/20], Loss: 0.1981
Epoch [9/20], Loss: 0.2416
Epoch [10/20], Loss: 0.4444
Epoch [11/20], Loss: 0.2375
Epoch [12/20], Loss: 0.1701
Epoch [13/20], Loss: 0.3165
Epoch [14/20], Loss: 0.1589
Epoch [15/20], Loss: 0.1565
Epoch [16/20], Loss: 0.1327
Epoch [17/20], Loss: 0.0645
Epoch [18/20], Loss: 0.1745
Epoch [19/20], Loss: 0.1516
Epoch [20/20], Loss: 0.1055
Accuracy on test data: 49.15%


Classes in train and test datasets are correctly mapped!
